In [1]:
%matplotlib qt
# %matplotlib ipympl

In [2]:
## Imports
print("Importing...")
import re
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import cv2
import easyocr

from matplotlib.backends.backend_pdf import PdfPages
# from matplotlib.widgets import Slider
import recognizer_modules


Importing...


In [3]:
########################################################
##########               INPUTS               ##########
########################################################

VIDEO_PATH = r"Examples\Not_processed-full_font\Video.avi"
# VIDEO_PATH = None

In [4]:
## Processor settings
rules = dict(re_rule=r'-?\d{1,3}\.\d', )


class Processor(recognizer_modules.ImageProcessor):
    Blur = range(1, 50)

    variables = [
        dict(name='Viscosity', rules=rules),
        dict(name='Temperature', rules=rules),
    ]

    def process(self, image):
        image = cv2.blur(image, (int(self['Blur']), int(self['Blur'])))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.bitwise_not(image)
        return image

In [5]:
## Global settings
if VIDEO_PATH is None:
    input_path = ''
    while input_path == '':
        input_path = input(f"Input video path: ")
    VIDEO_PATH = input_path

CAP = cv2.VideoCapture(VIDEO_PATH)
FPS = int(CAP.get(cv2.CAP_PROP_FPS))
LENTH = int(CAP.get(cv2.CAP_PROP_FRAME_COUNT) / FPS)
CAP.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, START_FRAME = CAP.read()

processor = Processor(CAP)
processor.configure_process()
processor.select_window()
processor.check_process()
print('Configurating end')

Configurate image processing
Configurating end


In [8]:
class Checker:

    def check(self, image, raw_value, rules):
        pattern_check = self.pattern_check(raw_value, **rules)
        if pattern_check is not None: return 0, pattern_check

        # img_check = self._image_check(image, rules)
        # if img_check is not None: return 1, img_check

        # value_check = self._value_check(raw_value)
        # if value_check is not None: return 2,value_check
        return 99, None

    def __init__(self, processor: recognizer_modules.ImageProcessor, reader):
        self.inner_processor = copy.deepcopy(processor)
        self._reader = reader

    def pattern_check(
        self,
        value: list,
        re_rule=None,
        min_rule=None,
        max_rule=None,
    ):
        if value == []: return None
        value = value[0]
        value = value.replace(',', '.')
        one_check = len(re.findall(re_rule, value)) == 1
        try:
            value = float(value)
        except ValueError:
            return None
        min_check = value <= min_rule if min_rule is not None else True
        max_check = value >= max_rule if max_rule is not None else True

        result = value if one_check and min_check and max_check else None
        return result

    def processor_configurator(self):
        for i in range(1, 50):
            self.inner_processor.blur = i
            yield True

    def processor_check(self, image, rules):
        configurator = self.processor_configurator()
        loop = True
        while loop:
            processed_img = self.inner_processor(image=image)
            raw_value = [
                value for _, value, _ in self._reader.readtext(processed_img)
            ]

            result = self.pattern_check(raw_value, **rules)
            if result is not None:
                return result
            else:
                try:
                    loop = configurator.__next__()
                except StopIteration:
                    return None

    def value_check(self, raw_value: list):
        parts = len(raw_value)
        if parts == 1:
            try:
                check_result = float(raw_value[0])
                if check_result > 1000:
                    raw_result = str(check_result)
                    raw_result = raw_result[:3] + '.' + raw_result[4]
            except:
                raw_result = None

        elif parts == 2:
            raw_result = '.'.join(raw_value)

        elif parts == 3:
            raw_result = f'{raw_value[0]}.{raw_value[2]}'

        try:
            result = float(raw_result)
        except:
            result = None
        return result


In [9]:
## Recognize
print('Starting recognizer...')
reader = easyocr.Reader(['en'])
checker = Checker(reader=reader, processor=processor)

input_fps = input('Input number of frames per second: ')
try:
    read_fps = float(input_fps)
except:
    read_fps = 1


print('Recognizing:')
errors = 0
frame_line = tqdm(iterable=range(0, FPS * LENTH, int(FPS / read_fps)))
frame_line.set_description(f'Errors: {errors: >4}')
data = []

for i_frame in frame_line:
    CAP.set(cv2.CAP_PROP_POS_FRAMES, i_frame)
    _, frame = CAP.read()
    i_text = {'time': round(i_frame / FPS, 1)}
    processed_frame= processor(frame)
    stricted_images = processor.strict(processed_frame)
    
    for variable in processor.variables:
        var_name = variable['name']
        var_rules = variable['rules']
        raw_value = [value for _, value, _ in reader.readtext(stricted_images[var_name])]

        mark, result = checker.check(image=frame,
                               raw_value=raw_value,
                               rules=var_rules)
        i_text[variable['name']] = result
        i_text[variable['name'] + '_mark'] = mark

    if None in i_text.values():
        errors += 1
        frame_line.set_description(f'Errors: {errors: >4}')
    data.append(i_text)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Starting recognizer...
Recognizing:


Errors:    0: 100%|██████████| 54/54 [00:18<00:00,  2.88it/s]


In [10]:
## Print
pd.DataFrame(data)

,time,Viscosity,Viscosity_mark,Temperature,Temperature_mark
0,0.0,241.9,0,17.4,0
1,1.0,241.4,0,17.4,0
2,2.0,240.9,0,17.4,0
3,3.0,240.4,0,17.4,0
4,4.0,239.9,0,17.4,0
5,5.0,239.4,0,17.4,0
6,6.0,239.9,0,17.4,0
7,7.0,240.4,0,17.4,0
8,8.0,240.4,0,17.4,0
9,9.0,240.9,0,17.4,0


In [ ]:
fig, ax = plt.subplots()
ax.plot([15, 18, 20, 22, 24, 26, 28, 30, 35, 40],
        [300, 225, 174, 152, 130, 117, 100, 77, 62.5, 45])
ax.plot(
    [15, 18, 20, 22, 24, 26, 28, 30, 35, 40],
    [189, 147, 126, 100, 98, 80, 72, 65, 45, 35.6],
)
ax.plot([15, 18, 20, 22, 24, 26, 28, 30, 65, 40],
        [150, 115, 100, 92, 80, 70, 60, 54, 40, 31])

ax.set_xlim(15.0, 40)
